In [1]:
'''
START HERE
'''
%pylab inline
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize()

Populating the interactive namespace from numpy and matplotlib
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
data = pd.read_csv('provider_data.csv')

/Users/mgbvox/opt/anaconda3/envs/ds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def consolidate_specialties(s):
    if specialty_map.get(s):
        return specialty_map[s]
    return s

similarSpecialtyDescrs = [("Dentist", "Oral & Maxillofacial Surgery"),
                          ("Radiology", "Radiologic Technologist"),
                          ("Assistant, Podiatric", "Podiatrist"),
                          ("Ophthalmology", "Optometrist"),
                          ("Physical Therapist", "Massage Therapist", "Physical Therapy Assistant",\
                           "Mechanotherapist", "Rehabilitation Practitioner", "Rehabilitation Hospital", "Chiropractor"),
                          ("Psychiatric Unit", "Psychoanalyst", "Psychiatric Hospital", "Psychiatry & Neurology", "Psychologist", "Behavioral Analyst"),
                          ("Pathology", "Technician, Pathology", "Spec/Tech, Pathology"),
                          ("Pain Medicine", "Nurse Anesthetist, Certified Registered",\
                           "Anesthesiologist Assistant", "Anesthesiology")]


specialty_map = dict()
for t in similarSpecialtyDescrs:
    first = t[0]
    remainder = t[1:]
    for val in remainder:
        specialty_map[val] = first
        
        
specialtyDescrs = ["Dermatology", "Otolaryngology", "Plastic Surgery", "Ophthalmology",
                   "Radiology", "Pathology",  "Physical Therapist" ]

paper_data = data[['npi','provider_type','hcpcs_code','bene_day_srvc_cnt']]

paper_data['provider_type'] = paper_data.provider_type.parallel_apply(consolidate_specialties)

/Users/mgbvox/opt/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
def in_desc(x):
    try:
        b = x in specialtyDescrs
        return b
    except:
        return False

In [5]:
provider_data = paper_data[paper_data['provider_type'].parallel_apply(in_desc)]

In [22]:
'''
Get codes that are relatively unique to single/small groups of provider types
'''

shared_hcpcs_threshold = 3

code_provider_crossover = data.groupby('hcpcs_code').provider_type.nunique()
provider_unique_codes = code_provider_crossover[code_provider_crossover<shared_hcpcs_threshold].index

provider_data = provider_data[provider_data.hcpcs_code.apply(lambda x: x in provider_unique_codes)]

In [23]:
provider_data['provider_type_oh'] = pd.get_dummies(provider_data.provider_type).parallel_apply(lambda x: np.array(x), axis=1)
provider_data['hcpcs_code_oh'] = pd.get_dummies(provider_data.hcpcs_code).parallel_apply(lambda x: np.array(x), axis=1)

In [24]:
procedures_by_discipline = provider_data.groupby('provider_type')['hcpcs_code'].unique()

In [25]:
def sum_oh(g):
    return np.stack(g, axis=0).sum(0)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

cpi_vec = provider_data.groupby('npi')['hcpcs_code_oh'].apply(sum_oh)

sims = cosine_similarity(pd.DataFrame(np.stack(cpi_vec.values, 0)))

cpi_vec = cpi_vec.reset_index()

In [27]:
import networkx as nx

similar_doc_idxs = np.where(sims > .8)
edges = np.array(list(zip(*similar_doc_idxs)))

#idxs = np.random.choice(len(edges), 1000)
#r_edgs = edges[idxs]

g = nx.Graph()

g.add_edges_from(edges)

In [28]:
!pip install tqdm

In [29]:
from tqdm import tqdm

In [30]:
npi_to_type = provider_data.groupby('npi')['provider_type'].unique()

In [31]:
personalization_dict = dict()
pos = .8
neg = 1-pos

provider_type = 'Ophthalmology'

for n in tqdm(list(g.nodes)):
    npi = cpi_vec.iloc[n]['npi']
    type_ = npi_to_type[npi][0]
    if type_ == provider_type:
        personalization_dict[n] = pos
    else:
        personalization_dict[n] = neg
        

100%|██████████| 6934/6934 [00:01<00:00, 4669.92it/s]


In [32]:
#For provider_type = 'Ophthalmology'

provider_pagerank = nx.pagerank(g, personalization=personalization_dict)

sorted_rank = sorted(provider_pagerank, key=lambda x: provider_pagerank[x])

In [33]:
result = []
for n, score in tqdm(provider_pagerank.items()):
    #type_ = cpi_vec.iloc[pid]['provider_type']
    npi = cpi_vec.iloc[n]['npi']
    type_ = npi_to_type[npi][0]
    result.append((score, type_, n, npi))

100%|██████████| 6934/6934 [00:01<00:00, 4554.53it/s]


In [34]:
opth = pd.DataFrame.from_records(sorted(result)[::-1])

In [35]:
opth.columns = ['ppr_score', 'specialty', 'node_idx', 'npi']

In [36]:
opth[opth.specialty != 'Ophthalmology']

,ppr_score,specialty,node_idx,npi
3043,0.000165,Pathology,1531,1225097686
3044,0.000165,Pathology,710,1104865146
3157,0.000150,Pathology,453,1063774214
3158,0.000150,Pathology,3827,1548436215
3184,0.000147,Otolaryngology,4361,1629164967
...,...,...,...,...
6929,0.000014,Pathology,6391,1922055144
6930,0.000014,Pathology,45,1003865197
6931,0.000013,Pathology,3909,1568434876
6932,0.000013,Pathology,2872,1417967761


In [39]:
cpi_vec.iloc[1531]['npi']

1225097686

In [53]:
for ridx, d in data[data.npi == 1225097686].iterrows():
    if d['hcpcs_code'] in provider_unique_codes:
        print(d['hcpcs_description'])

Pap test (Pap smear)
Evaluation of fine needle aspirate with interpretation and report
Evaluation of fine needle aspirate with interpretation and report
Pathology examination of tissue specimen during surgery
